In [1]:
import pandas as pd
import os
from glob import glob
import requests
import zipfile

In [2]:
#Setup paths
currentWorkDir = os.getcwd()

In [3]:
# URL of the file
url = 'https://www.kaggle.com/api/v1/datasets/download/dscclass/phishing'

# Destination path
output_path = f'{currentWorkDir}/dados/phishing.zip'

# Send GET request with redirect support
response = requests.get(url, allow_redirects=True)

# Save the content to the file
with open(output_path, "wb") as f:
    f.write(response.content)

print("Download complete!")


Download complete!


In [4]:
extract_to = f'{currentWorkDir}/dados/Raw'

os.makedirs(extract_to, exist_ok=True)


with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to {extract_to}")

Files extracted to /home/lucasjansen/Desktop/SG_atividade3/dados/Raw


In [ ]:
# Primeiramente vamos importar as bibliotecas usadas neste estudo de caso como segue.
import numpy as np #biblioteca com recursos para álgebra linear
import pandas as pd # processamento dos dados, carga de arquivo CSV e de I/O
import json
from sklearn import * 
import sklearn.model_selection as model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


#import matplotlib.pyplot as plt
#from collections import Counter
#from sklearn.cross_validation import train_test_split

In [5]:
# Nós utilizaremos neste caso, o dataset (conjunto de dados) phishing.csv, disponível aqui pra você.
# A origem deste arquivo csv foi 'https://archive.ics.uci.edu/ml/datasets/phishing+websites'
#
# Primeiro, vamos testar o carregamento do arquivo CSV.
# Não vamos usar com detalhes a biblioteca pandas neste caso, mas esse trecho de código ajuda a entender o arquivo com o conjunto de dados.
df = pd.read_csv(f'{currentWorkDir}/dados/Raw/phishing.csv')
df.info() #apresenta os metadados do arquivo csv
df.tail(10)
#df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054 entries, 0 to 11053
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   -1      11054 non-null  int64
 1   1       11054 non-null  int64
 2   1.1     11054 non-null  int64
 3   1.2     11054 non-null  int64
 4   -1.1    11054 non-null  int64
 5   -1.2    11054 non-null  int64
 6   -1.3    11054 non-null  int64
 7   -1.4    11054 non-null  int64
 8   -1.5    11054 non-null  int64
 9   1.3     11054 non-null  int64
 10  1.4     11054 non-null  int64
 11  -1.6    11054 non-null  int64
 12  1.5     11054 non-null  int64
 13  -1.7    11054 non-null  int64
 14  1.6     11054 non-null  int64
 15  -1.8    11054 non-null  int64
 16  -1.9    11054 non-null  int64
 17  -1.10   11054 non-null  int64
 18  0       11054 non-null  int64
 19  1.7     11054 non-null  int64
 20  1.8     11054 non-null  int64
 21  1.9     11054 non-null  int64
 22  1.10    11054 non-null  int64
 23  -1.11   110

,-1,1,1.1,1.2,-1.1,-1.2,-1.3,-1.4,-1.5,1.3,...,1.9,1.10,-1.11,-1.12,-1.13,-1.14,1.11,1.12,-1.15,-1.16
11044,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,0,-1,1,0,1,1
11045,-1,-1,1,1,1,-1,1,1,-1,1,...,1,1,1,1,0,-1,1,1,1,1
11046,-1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1
11047,1,-1,1,1,1,-1,-1,1,1,1,...,1,1,1,1,0,-1,1,0,1,1
11048,-1,-1,1,1,-1,-1,1,-1,1,1,...,1,1,-1,1,1,-1,1,1,1,-1
11049,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11050,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11051,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1
11052,-1,-1,1,1,1,-1,-1,-1,1,-1,...,-1,1,1,1,1,-1,1,1,1,-1
11053,-1,-1,1,1,1,-1,-1,-1,1,1,...,1,1,-1,1,-1,-1,-1,1,-1,-1


In [ ]:
# Vamos agora carregar o dataset que iremos utilizar na variável np.
training_data = np.genfromtxt(f'{currentWorkDir}/dados/Raw/phishing.csv', delimiter=',', dtype=np.int32)
print(training_data) #para ver os dados na variável.

In [ ]:
# Identificando as entradas (todos os atributos, exceto o último) e as saídas (o último atributo):
inputs = training_data[:,:-1]
outputs = training_data[:,-1]

print(inputs.shape)
print(outputs.shape)

In [ ]:
# Dividindo o dataset em treino e teste. 
#training_inputs = inputs[:2000]
#training_outputs = outputs[:2000]
#testing_inputs = inputs[2000:]
#testing_outputs = outputs[2000:]

# dividing the dataset into training and testing:
training_inputs, testing_inputs, training_outputs, testing_outputs = model_selection.train_test_split(inputs, outputs, test_size=0.2)

print(training_inputs.shape)
print(testing_inputs.shape)
print(training_outputs.shape)
print(testing_outputs.shape)


In [ ]:
# Preparando o classificador Scikit-learn do tipo regressão logística.
classifier1 = LogisticRegression()

In [ ]:
# Preparando o classificador Scikit-learn do tipo árvore de decisão. 
classifier2 = DecisionTreeClassifier()

In [ ]:
# Treinando os classificadores.
classifier1.fit(training_inputs, training_outputs)
classifier2.fit(training_inputs, training_outputs)

In [ ]:
# Fazendo as predições. 
predictions1 = classifier1.predict(testing_inputs)
predictions2 = classifier2.predict(testing_inputs)

In [ ]:
# Calculando a acurácia para o modelo de detecção de phishing.
accuracy1 = 100.0 * accuracy_score(testing_outputs, predictions1)
accuracy2 = 100.0 * accuracy_score(testing_outputs, predictions2)

In [ ]:
# Imprimindo a acurácia para o modelo de detecção de phishing.
print ("A acurácia para o classificador de regressão logística sobre os dados de teste é: " +str(accuracy1))
print ("A acurácia para o classificador de árvore de decisão sobre os dados de teste é: " +str(accuracy2))